## pyJvsip FFT Usage
In this document we cover the FFT support for the discrete Fourier transform in pyJvsip. Although FFT is a class, we also define some convienience methods for views. 

In [1]:
import pyJvsip as pjv

### Argument List for Fast Fourier Transform routines
Generally to create an FFT object you need length(s), a scale factor, an FFT direction (forward or inverse), major direction either by row or by column (for matrices), and two hints the first `ntimes` indicating how often you think the FFT object will be used, and the second an algorithm hint indicating if you want the object to be optimized for time of execution, memory space used, or accuracy of answer. 

In this document for vectors we indicate the length as `lnth`. For matrices we use `M` for column length and `N` for row length.

We note that for a real to complex FFT there is no direction argument since this is always a forward FFT. Similarily for a complex to real FFT there is also no direction argument since this is always an inverse FFT.

### Hints
In General, other than supporting the interface, JVSIP does not support algorithm hints. For FFT this means the hint argument and the number of times argument have no affect. Generally I just use zero for ntimes, and VSIP_ALG_TIME for aglorithm hint.  Note that if you are trying to write portable code then you should select these items bassed upon your need. For instance for view methods I selected 1 for number of times since these are always a one-off FFT.

### Type
For pyJvsip types are defined when creating objects using a type string. The type strings used below were defined for FFTs.

In this document the examples are done for double. To get single float just replace the `_d` at the end of the type string with `_f`.

### FFT object creation
Below we create FFT objects and define some parameters. The scale argument is defined as 1 for the forward FFT and as (iscale) 1.0/lnth for the inverse so that when we do the inverse FFT on the output of the forward FFT we get back the origional data.

I have tried to do an example of every fft type, but it is possible I missed one or two.  Basically the argument tuple is the same as the function call defined for fft and fftm creates in the C VSIPL specificaiton. The type string before the argument tuple controls the underlying C VSIPL object created by the FFT object initializtion. If the string does not match the argument list there will be trouble. The C VSIPL object is contained as an instance variable in the pyJvsip FFT object.

In [2]:
lnth=1024 # for vectors
M = 64  # column length
N = 128 # row length
fwd=pjv.VSIP_FFT_FWD
inv=pjv.VSIP_FFT_INV
hint=pjv.VSIP_ALG_TIME
ntimes=0
scale=1.0
iscale=1.0/lnth
row=pjv.VSIP_ROW
col=pjv.VSIP_COL
#fft
argrcfft=(lnth,scale,ntimes,hint)
objrc=pjv.FFT('rcfftop_d',*argrcfft)
argcrfft=(lnth,iscale,ntimes,hint) #change scale to remove FFT amplification
objcr=pjv.FFT('crfftop_d',*argcrfft)
argccfft=(lnth,scale,fwd,ntimes,hint)
objccfwd=pjv.FFT('ccfftop_d',*argccfft)
objccipfwd=pjv.FFT('ccfftip_d',*argccfft)
argccfft=(lnth,iscale,inv,ntimes,hint)
objccinv=pjv.FFT('ccfftop_d',*argccfft)
objccipinv=pjv.FFT('ccfftip_d',*argccfft)
#fftm by row
iscale=1.0/N
argrcfftm=(M,N,scale,row,ntimes,hint)
objrcmr=pjv.FFT('rcfftmop_d',*argrcfftm)
argcrfftm=(M,N,iscale,row,ntimes,hint)
objcrmr=pjv.FFT('crfftmop_d',*argcrfftm)
argccfftm=(M,N,scale,fwd,row,ntimes,hint)
objccfwdmr=pjv.FFT('ccfftmop_d',*argccfftm)
objccipfwdmr=pjv.FFT('ccfftmip_d',*argccfftm)
argccfftm=(M,N,iscale,inv,row,ntimes,hint)
objccinvmr=pjv.FFT('ccfftmop_d',*argccfftm)
objccipinvmr=pjv.FFT('ccfftmip_d',*argccfftm)
#fftm by col
iscale=1.0/M
argrcfftm=(M,N,scale,col,ntimes,hint)
objrcmc=pjv.FFT('rcfftmop_d',*argrcfftm)
argcrfftm=(M,N,iscale,col,ntimes,hint)
objcrmc=pjv.FFT('crfftmop_d',*argcrfftm)
argccfftm=(M,N,scale,fwd,col,ntimes,hint)
objccfwdmc=pjv.FFT('ccfftmop_d',*argccfftm)
objccipfwdmc=pjv.FFT('ccfftmip_d',*argccfftm)
argccfftm=(M,N,iscale,inv,col,ntimes,hint)
objccinvmc=pjv.FFT('ccfftmop_d',*argccfftm)
objccipinvmc=pjv.FFT('ccfftmip_d',*argccfftm)

#### FFT Use
Below we use the FFT objects created above to do some FFT's. Basically we create appropriate views and fill them with random data. We do a Forward FFT and an inverse FFT with appropriate scalling so we should get back the input data. We then compare using the Frobenious norm to check.

In [3]:
#Ex rcfftop and crfftop
assert lnth & 1==0,'for rcfft the vector must be unit stride and even length'
x=pjv.create('vview_d',lnth).randn(5)
y=pjv.create('cvview_d',int(lnth/2)+1).fill(0.0)
xc=x.copy
objrc.dft(x,y)
objcr.dft(y,x)
print('%f'%(x-xc).normFro)

0.000000


In [4]:
objrc.type

'rcfftop_d'

In [5]:
objcr.type

'crfftop_d'

In [6]:
objcr.arg

(1024, 0.0009765625, 0, 1)

In [7]:
#Ex ccfftop
x=pjv.create('cvview_d',lnth).randn(5)
y=pjv.create('cvview_d',lnth).fill(0.0)
xc=x.copy
objccfwd.dft(x,y)
objccinv.dft(y,x)
print('%f'%(x-xc).normFro)

0.000000


In [8]:
#Ex ccfftip
x=pjv.create('cvview_d',lnth).randn(5)
xc=x.copy
objccipfwd.dft(x)
objccipinv.dft(x)
print('%f'%(x-xc).normFro)

0.000000


In [10]:
#Ex rcfftmop and crfftmop
x=pjv.create('mview_d',M,N).randn(5) #this is row major
y=pjv.create('cmview_d',M,int(N/2)+1).fill(0.0) # do by row
xc=x.copy
objrcmr.dft(x,y)
objcrmr.dft(y,x)
print('%f'%(x-xc).normFro)
#Ex rcfftmop and crfftmop
x=pjv.create('mview_d',M,N,'COL').randn(5) #this is column major
y=pjv.create('cmview_d',int(M/2)+1,N).fill(0.0) # do by column
xc=x.copy
objrcmc.dft(x,y)
objcrmc.dft(y,x)
print('%f'%(x-xc).normFro)

0.000000
0.000000


In [11]:
#Ex ccfftmop by row
x=pjv.create('cmview_d',M,N).randn(5) #this is row major
y=pjv.create('cmview_d',M,N).fill(0.0) # do by row
xc=x.copy
objccfwdmr.dft(x,y)
objccinvmr.dft(y,x)
print('%f'%(x-xc).normFro)
#Ex ccfftmop by column
x=pjv.create('cmview_d',M,N,'COL').randn(5) #this is column major
y=pjv.create('cmview_d',M,N).fill(0.0) # do by column
xc=x.copy
objccfwdmc.dft(x,y)
objccinvmc.dft(y,x)
print('%f'%(x-xc).normFro)

0.000000
0.000000


In [12]:
#Ex ccfftmip by row
x=pjv.create('cmview_d',M,N).randn(5) #this is row major
xc=x.copy
objccipfwdmr.dft(x)
objccipinvmr.dft(x)
print('%f'%(x-xc).normFro)
#Ex ccfftmip by col
x=pjv.create('cmview_d',M,N,'COL').randn(5) #this is column major
xc=x.copy
objccipfwdmc.dft(x)
objccipinvmc.dft(x)
print('%f'%(x-xc).normFro)

0.000000
0.000000


### View Methods
In pyJvsip I supply some convenience methods for Views to do FFTs.  This is only for pyJvsip and has nothing to do with the C VSIPL specification. The FFT class is used to write the methods so they are not independent of the JVSIP C library.

View methods for fft are defined as properties (@property) and they have no arguments. This means the scale factor is determined by the implementation. I chose to just use one. In addition the selection of VSIP_FFT_FWD and VSIP_FFT_INV is done by the API. See examples below for naming information.

The view methods that are done out-of-place will create a suitable output block and view for the output data and return it.

Additional functionality for fftop has been added so that a real float vector will be treated as if it were a complex vector with zero imaginary part.

In the examples below for matrix views note the use of the view major attribute. These will default to ROW major but if the major attribute has been set to COL somewhere then that is the direction of the multiple FFT so it is wise to set it just before calling the function if you don't already know what it is.

In [13]:
# demonstrate rcfftop
x0=pjv.create('vview_d',lnth).randn(5)
y=x0.rcfft
# note for view methods scale is always 1.0
x=y.crfft
x *=1.0/lnth
print('%f'%(x-x0).normFro)

0.000000


In [15]:
x0=pjv.create('cvview_d',lnth).randn(5)
y=x0.fftop
# note we designate inverse with an i in front of the fft method name
x=y.ifftop * float(1.0/lnth)
print('%f'%(x-x0).normFro)

0.000000


In [ ]:
x0=pjv.create('cvview_d',lnth).randn(5)
x=x0.copy
x0.fftip
x0.ifftip
x0*=1.0/lnth
print('%f'%(x-x0).normFro)

In [ ]:
x0=pjv.create('cmview_d',15,26).randn(6)
x=x0.copy
x0.COL.fftip
x0.COL.ifftip
x0 *= 1.0/15
print('%f'%(x-x0).normFro)

In [ ]:
x0=pjv.create('cmview_d',15,26).randn(6)
x=x0.copy
x0.ROW.fftip
x0.ROW.ifftip
x0 *= 1.0/26
print('%f'%(x-x0).normFro)

In [ ]:
x0=pjv.create('cmview_d',15,26).randn(6)
x=x0.copy
y=x0.COL.fftop
z=y.COL.ifftop
z *= 1.0/15
print('%f'%(x-z).normFro)

In [ ]:
x0=pjv.create('cmview_d',15,26).randn(6)
x=x0.copy
y=x0.ROW.fftop
z=y.ROW.ifftop
z *= 1.0/26
print('%f'%(x-z).normFro)

In [ ]:
x0=pjv.create('mview_d',16,32,'COL').randn(6)
x=x0.copy
y=x0.COL.rcfft
z=y.COL.crfft
z *= 1.0/16
print('%f'%(x-z).normFro)

In [ ]:
# x0=pjv.create('mview_d',16,32,'ROW').randn(6)
x=x0.copy
y=x0.ROW.rcfft
z=y.ROW.crfft
z *= 1.0/32
print('%f'%(x-z).normFro)